In [1]:
!pip install qrcode

import qrcode
from PIL import Image, ImageDraw
from IPython.display import display

def show_qr_with_logo(
    url: str,
    logo_path: str,
    out_path: str = "qr_with_logo.png",
    *,
    box_size: int = 10,
    border: int = 4,
    logo_scale: float = 0.20,     # diameter ~20% of QR side
    border_ratio: float = 0.10,   # ring/frame thickness ~10% of logo diameter
    border_style: str = "ring"    # "ring" | "frame" | "none"
) -> None:
    """
    Generate, display, and save a QR code with a circular logo and optional white 'ring' or full 'frame'.

    border_style:
      - "ring": circular white ring around the logo (good contrast, minimal coverage)
      - "frame": solid white circular background behind logo (max separation, most robust)
      - "none": no white surround (use only if logo already contrasts well)
    """
    # 1) Build QR with high error correction
    qr = qrcode.QRCode(
        version=None,
        error_correction=qrcode.constants.ERROR_CORRECT_H,
        box_size=box_size,
        border=border,
    )
    qr.add_data(url)
    qr.make(fit=True)

    qr_img = qr.make_image(fill_color="black", back_color="white").convert("RGB")
    qr_size = qr_img.size[0]

    # 2) Clamp scales
    logo_scale = max(0.10, min(0.35, float(logo_scale)))
    border_ratio = max(0.04, min(0.25, float(border_ratio)))

    logo_diam = int(qr_size * logo_scale)
    if logo_diam % 2: logo_diam += 1
    border_px = max(2, int(round(logo_diam * border_ratio)))

    # 3) Load & circularize logo
    logo = Image.open(logo_path).convert("RGBA")
    logo = logo.resize((logo_diam, logo_diam), Image.LANCZOS)

    circle_mask = Image.new("L", (logo_diam, logo_diam), 0)
    ImageDraw.Draw(circle_mask).ellipse((0, 0, logo_diam, logo_diam), fill=255)

    circular_logo = Image.new("RGBA", (logo_diam, logo_diam))
    circular_logo.paste(logo, (0, 0), circle_mask)

    # 4) Build surround (ring/frame/none)
    border_style = border_style.lower().strip()
    if border_style == "none":
        composite = circular_logo
        outer = logo_diam
    else:
        outer = logo_diam + 2 * border_px
        composite = Image.new("RGBA", (outer, outer), (255, 255, 255, 0))
        # Full white disk
        full_mask = Image.new("L", (outer, outer), 0)
        ImageDraw.Draw(full_mask).ellipse((0, 0, outer, outer), fill=255)
        white_disk = Image.new("RGBA", (outer, outer), (255, 255, 255, 255))

        if border_style == "frame":
            # solid white circle background
            composite.paste(white_disk, (0, 0), full_mask)
            composite.paste(circular_logo, (border_px, border_px), circular_logo)
        elif border_style == "ring":
            # ring = outer white disk minus inner hole
            composite.paste(white_disk, (0, 0), full_mask)
            # punch inner hole by pasting transparent disk
            hole_mask = Image.new("L", (outer - 2*border_px, outer - 2*border_px), 255)
            hole = Image.new("RGBA", (outer - 2*border_px, outer - 2*border_px), (255, 255, 255, 0))
            composite.paste(hole, (border_px, border_px), hole_mask)
            # paste logo inside the ring
            composite.paste(circular_logo, (border_px, border_px), circular_logo)
        else:
            raise ValueError('border_style must be "ring", "frame", or "none".')

    # 5) Paste centered
    pos = ((qr_size - composite.size[0]) // 2, (qr_size - composite.size[1]) // 2)
    qr_img.paste(composite, pos, composite)

    # 6) Save & display
    qr_img.save(out_path)
    display(qr_img)
    print(f"Saved to {out_path}")

# Examples (run one at a time in your notebook):
show_qr_with_logo("https://www.linkedin.com/groups/5081467/", "./uwecyber.png", border_style="ring")
# show_qr_with_logo("https://www.uwe.ac.uk/study/work-experience-jobs-and-placements/placements", "./uwecyber.png", border_style="frame")
# show_qr_with_logo("https://www.linkedin.com/groups/5081467/", "my_logo.png", border_style="none")


FileNotFoundError: [Errno 2] No such file or directory: './uwecyber.png'